# VacationPy
---
## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,san isidro,-12.1167,-77.0500,19.77,73,75,4.63,PE,1697139714
1,1,port-aux-francais,-49.3500,70.2167,2.10,78,82,7.10,TF,1697139715
2,2,arkhangel'sk,64.5401,40.5433,7.85,92,65,4.41,RU,1697139715
3,3,lesnoy,57.6198,63.0784,2.16,93,100,7.38,RU,1697139715
4,4,barcelos,41.5388,-8.6151,24.95,51,100,1.89,PT,1697139716


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot
map1 = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo=True, 
    tiles="OSM",
    size = "Humidity",
    frame_width = 800,
    frame_height = 600,
    color = "City",
    alpha = 0.75)

# Display the map
map1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df["Max Temp"]<24) & (city_data_df["Max Temp"]>17) 
                               & (city_data_df["Humidity"]<70) & (city_data_df["Wind Speed"]<4.5) 
                                & (city_data_df["Cloudiness"]==0)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,20,bethel,41.3712,-73.4140,21.78,46,0,3.60,US,1697139720
107,107,turpan,42.9333,89.1667,21.28,21,0,1.09,CN,1697139744
118,118,villa huidobro,-34.8383,-64.5869,21.76,14,0,3.84,AR,1697139747
163,163,keren,15.7778,38.4581,21.43,65,0,3.30,ER,1697139759
209,209,selma,36.5708,-119.6121,23.46,45,0,3.09,US,1697139771
251,251,qurayyat,31.3318,37.3428,23.10,31,0,0.00,SA,1697139782
379,379,thala,35.5724,8.6703,18.79,31,0,2.38,TN,1697139817
443,443,zhob,31.3411,69.4481,19.56,19,0,0.87,PK,1697139833
530,530,pisco,-13.7000,-76.2167,22.03,64,0,3.60,PE,1697139755


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the 
# city, country, coordinates, and humidity.
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
20,bethel,US,41.3712,-73.4140,46,
107,turpan,CN,42.9333,89.1667,21,
118,villa huidobro,AR,-34.8383,-64.5869,14,
163,keren,ER,15.7778,38.4581,65,
209,selma,US,36.5708,-119.6121,45,
251,qurayyat,SA,31.3318,37.3428,31,
379,thala,TN,35.5724,8.6703,31,
443,zhob,PK,31.3411,69.4481,19,
530,pisco,PE,-13.7000,-76.2167,64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000
params = {"apiKey": geoapify_key,
         "categories": "accommodation.hotel"}

# print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get lat & lon from df
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make an API request using the params dictionary & convert to json
    name_address = requests.get(base_url, params=params).json()

    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
turpan - nearest hotel: Oriental
villa huidobro - nearest hotel: No hotel found
keren - nearest hotel: Eritrea Hotel قندق إرتريا
selma - nearest hotel: Best Western
qurayyat - nearest hotel: دانة الشمال
thala - nearest hotel: No hotel found
zhob - nearest hotel: No hotel found
pisco - nearest hotel: La Portada


,City,Country,Lat,Lng,Humidity,Hotel Name
20,bethel,US,41.3712,-73.4140,46,Hampton Inn Danbury
107,turpan,CN,42.9333,89.1667,21,Oriental
118,villa huidobro,AR,-34.8383,-64.5869,14,No hotel found
163,keren,ER,15.7778,38.4581,65,Eritrea Hotel قندق إرتريا
209,selma,US,36.5708,-119.6121,45,Best Western
251,qurayyat,SA,31.3318,37.3428,31,دانة الشمال
379,thala,TN,35.5724,8.6703,31,No hotel found
443,zhob,PK,31.3411,69.4481,19,No hotel found
530,pisco,PE,-13.7000,-76.2167,64,La Portada


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
# Configure the map plot
map2 = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo=True, 
    tiles="OSM",
    size = "Humidity",
    frame_width = 800,
    frame_height = 600,
    color = "City",
    alpha = 0.75,
    hover_cols=["Hotel Name", "Country"])

# Display the map
map2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)